# Apply this code to sample TS from model.

In [1]:
# --- Importing and defining some functions ----
import os
import torch
import py3Dmol
import numpy as np

from typing import Optional
from torch import tensor
from e3nn import o3
from torch_scatter import scatter_mean

from oa_reactdiff.model import LEFTNet

default_float = torch.float64
torch.set_default_dtype(default_float)  # Use double precision for more accurate testing


def remove_mean_batch(
    x: tensor, 
    indices: Optional[tensor] = None
) -> tensor:
    """Remove the mean from each batch in x

    Args:
        x (tensor): input tensor.
        indices (Optional[tensor], optional): batch indices. Defaults to None.

    Returns:
        tensor: output tensor with batch mean as 0.
    """
    if indices == None:
         return x - torch.mean(x, dim=0)
    mean = scatter_mean(x, indices, dim=0)
    x = x - mean[indices]
    return x


def draw_in_3dmol(mol: str, fmt: str = "xyz") -> py3Dmol.view:
    """Draw the molecule

    Args:
        mol (str): str content of molecule.
        fmt (str, optional): format. Defaults to "xyz".

    Returns:
        py3Dmol.view: output viewer
    """
    viewer = py3Dmol.view(1024, 576)
    viewer.addModel(mol, fmt)
    viewer.setStyle({'stick': {}, "sphere": {"radius": 0.36}})
    viewer.zoomTo()
    return viewer


def assemble_xyz(z: list, pos: tensor) -> str:
    """Assembling atomic numbers and positions into xyz format

    Args:
        z (list): chemical elements
        pos (tensor): 3D coordinates

    Returns:
        str: xyz string
    """
    natoms =len(z)
    xyz = f"{natoms}\n\n"
    for _z, _pos in zip(z, pos.numpy()):
        xyz += f"{_z}\t" + "\t".join([str(x) for x in _pos]) + "\n"
    return xyz

### Building a LEFTNet model

A simple test is performed to verify SE(3) symmetry. The model here is for testing, so we only need to build a very small model.

Note: [LEFTNet](https://arxiv.org/abs/2304.04757) is a new SOTA-level SE(3) graph neural network. Although we use LEFTNet here, the properties it exhibits are model-independent (other SE(3) models, such as [EGNN](https://arxiv.org/pdf/2102.09844.pdf), will give the same results)

TL: EGNN is not SE$(3)$  equivariant?

In [2]:
num_layers = 2
hidden_channels = 8
in_hidden_channels = 4
num_radial = 4

model =  LEFTNet(
    num_layers=num_layers,
    hidden_channels=hidden_channels,
    in_hidden_channels=in_hidden_channels,
    num_radial=num_radial,
    object_aware=False,
)

sum(p.numel() for p in model.parameters() if p.requires_grad)

/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


7882



### Create an "Object-Aware" LEFTNet

In [3]:
# --- Importing necessary function ---
from torch.utils.data import DataLoader

from oa_reactdiff.trainer.pl_trainer import DDPMModule


from oa_reactdiff.dataset import ProcessedTS1x, ProcessedSCAN
from oa_reactdiff.diffusion._schedule import DiffSchedule, PredefinedNoiseSchedule

from oa_reactdiff.diffusion._normalizer import FEATURE_MAPPING
from oa_reactdiff.analyze.rmsd import batch_rmsd

from oa_reactdiff.utils.sampling_tools import assemble_sample_inputs, write_tmp_xyz

In [4]:
!pwd

/misc/home/guest50/OAReactDiff



### Import the pre-trained model and redefine the schedule.

In [5]:
# TL fix: {
from oa_reactdiff.trainer.pl_trainer import DDPMModule
# } fix. Why didn' this carry over from the previous cell import statement?

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")
print(device) # TL

tspath = os.path.abspath(os.path.join(os.getcwd(), "oa_reactdiff","trainer"))
print(tspath)
# zenodo_pretrained_ckpt
ddpm_trainer = DDPMModule.load_from_checkpoint(
    checkpoint_path="./trained_models/pretrained-SCAN-diff-2-wo_selfloops.ckpt",
    #checkpoint_path=os.path.abspath("/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer/checkpoint/OAReactDiff/leftnet-0-84724de349da/ddpm-epoch=1964-val-totloss=510.42.ckpt"),
    #checkpoint_path=os.path.abspath("/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer/checkpoint/OAReactDiff/leftnet-0-84724de349da/ddpm-epoch=1999-val-totloss=523.77.ckpt"),
    #checkpoint_path="./pretrained-ts1x-diff.ckpt", # original
    #checkpoint_path=f"{tspath}/checkpoint/OAReactDiff/leftnet-0-f1ff7dc18fa3/ddpm-epoch=1999-val-totloss=509.31.ckpt", # Our recapitulation
    map_location=device,
)
ddpm_trainer = ddpm_trainer.to(device)

cuda
/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer


/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


In [6]:
noise_schedule: str = "polynomial_2"
timesteps: int = 150
precision: float = 1e-5

gamma_module = PredefinedNoiseSchedule(
            noise_schedule=noise_schedule,
            timesteps=timesteps,
            precision=precision,
        )
schedule = DiffSchedule(
    gamma_module=gamma_module,
    norm_values=ddpm_trainer.ddpm.norm_values
)
ddpm_trainer.ddpm.schedule = schedule
ddpm_trainer.ddpm.T = timesteps
ddpm_trainer = ddpm_trainer.to(device)


### Prepare dataset and data loader and select a reaction involving multiple molecules

In [7]:
import pickle

npz_path = "./oa_reactdiff/data/SCAN-third/train.pkl" # w/o self-loops after Angstrom correction

train_pkl = pickle.load(open(npz_path, "rb"))

In [8]:
train_pkl["use_ind"][:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
train_pkl["reactant"]["rxn"][177]

'ALD1-TS196'

In [10]:
train_pkl["reactant"]['num_atoms'][177]

17

In [11]:
dataset = ProcessedSCAN(
    npz_path=npz_path,
    center=True,
    pad_fragments=0,
    device=device,
    zero_charge=False,
    remove_h=False,
    single_frag_only=False,
    swapping_react_prod=False,
    use_by_ind=True,
)
loader = DataLoader(
    dataset, 
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=dataset.collate_fn
)
itl = iter(loader)
idx = -1 # TL: why?

len(dataset)

87738

In [12]:
num_indices_to_select = 20
SEED = 747
np.random.default_rng(seed=SEED)
np.random.seed(SEED)
random_indices = np.random.choice(len(dataset), size=num_indices_to_select, replace=False)
random_indices = sorted(random_indices)
random_indices

[85,
 582,
 1030,
 14456,
 18657,
 19738,
 21233,
 27343,
 27677,
 32299,
 37392,
 41209,
 42208,
 51709,
 53554,
 57756,
 64958,
 65787,
 80489,
 87011]

In [13]:
train_indices = [train_pkl["use_ind"][x] for x in random_indices]
train_indices

[92,
 636,
 1131,
 16119,
 20816,
 22024,
 23683,
 30463,
 30832,
 35956,
 41618,
 45847,
 46957,
 57472,
 59519,
 64145,
 72169,
 73095,
 89393,
 96696]

In [14]:
# TL visualize each state:
atomic_num2sym = {
    1: 'H',    2: 'He',   3: 'Li',   4: 'Be',   5: 'B',    6: 'C',    7: 'N',    8: 'O',    9: 'F',    10: 'Ne',
    11: 'Na',  12: 'Mg',  13: 'Al',  14: 'Si',  15: 'P',   16: 'S',   17: 'Cl',  18: 'Ar',  19: 'K',   20: 'Ca',
    21: 'Sc',  22: 'Ti',  23: 'V',   24: 'Cr',  25: 'Mn',  26: 'Fe',  27: 'Co',  28: 'Ni',  29: 'Cu',  30: 'Zn',
    31: 'Ga',  32: 'Ge',  33: 'As',  34: 'Se',  35: 'Br',  36: 'Kr',  37: 'Rb',  38: 'Sr',  39: 'Y',   40: 'Zr',
    41: 'Nb',  42: 'Mo',  43: 'Tc',  44: 'Ru',  45: 'Rh',  46: 'Pd',  47: 'Ag',  48: 'Cd',  49: 'In',  50: 'Sn',
    51: 'Sb',  52: 'Te',  53: 'I',   54: 'Xe',  55: 'Cs',  56: 'Ba',  57: 'La',  58: 'Ce',  59: 'Pr',  60: 'Nd',
    61: 'Pm',  62: 'Sm',  63: 'Eu',  64: 'Gd', 65: 'Tb',  66: 'Dy',  67: 'Ho',  68: 'Er',  69: 'Tm',  70: 'Yb',
    71: 'Lu',  72: 'Hf',  73: 'Ta',  74: 'W',   75: 'Re',  76: 'Os',  77: 'Ir',  78: 'Pt',  79: 'Au',  80: 'Hg',
    81: 'Tl',  82: 'Pb',  83: 'Bi',  84: 'Po',  85: 'At',  86: 'Rn',  87: 'Fr',  88: 'Ra',  89: 'Ac',  90: 'Th',
    91: 'Pa',  92: 'U',   93: 'Np',  94: 'Pu',  95: 'Am',  96: 'Cm',  97: 'Bk',  98: 'Cf',  99: 'Es', 100: 'Fm',
    101: 'Md', 102: 'No', 103: 'Lr', 104: 'Rf', 105: 'Db', 106: 'Sg', 107: 'Bh', 108: 'Hs', 109: 'Mt', 110: 'Ds',
    111: 'Rg', 112: 'Cn', 113: 'Nh', 114: 'Fl', 115: 'Mc', 116: 'Lv', 117: 'Ts', 118: 'Og'
}

In [15]:
def xyz_block_from_node_features(xh: torch.tensor, comment: str="", c2a: dict=atomic_num2sym) -> str:
    num_atoms = xh.shape[0]
    xyz_lines = [str(num_atoms), comment]
    for row in xh:
        position = row[:3].cpu().numpy()
        z = c2a[row[-1].long().item()]
        xyz_lines.append(f"{z}\t" + "\t".join([str(x) for x in position]))
    return "\n".join(xyz_lines)

In [16]:
!ls -haltr results

total 24K
drwxr-xr-x  2 guest50 users 4.0K Jun 11 17:49 sample_20_TS-20250611-Tr1x-from_scratch_pretrained_ckpt
drwxr-xr-x  2 guest50 users 4.0K Jun 11 18:05 sample_20_TS-20250611-Tr1x-zenodo_pretrained_ckpt
drwxr-xr-x  2 guest50 users 4.0K Jun 11 18:57 sample_20_TS-20250611-SCAN-from_scratch_pretrained_ckpt
drwxr-xr-x  2 guest50 users 4.0K Jun 17 18:13 sample_20_TS-20250616-SCAN-w_selfloops-from_scratch_pretrained_ckpt
drwxr-xr-x  2 guest50 users   10 Jun 17 18:20 sample_20_TS-20250616-SCAN-wo_selfloops-from_scratch_pretrained_ckpt
drwxr-xr-x  7 guest50 users 4.0K Jun 17 18:20 .
drwxr-xr-x 15 guest50 users 4.0K Jun 17  2025 ..


In [17]:
output_dir = os.path.abspath("results/sample_20_TS-20250616-SCAN-wo_selfloops-from_scratch_pretrained_ckpt")
#output_dir = None

In [18]:
selected_representation_triples = {}
rs_rmsdsx = []
ts_rmsdsx = []
ps_rmsdsx = []

for i in range(len(dataset)): 
    representations, res = next(itl)
    if i in random_indices:
        xyz_blocks = []
        print(i)
        train_idx = train_pkl["use_ind"][i]
        print(train_idx)
        rxn_id = train_pkl["reactant"]["rxn"][train_idx]
        n_samples = representations[0]["size"].size(0)
        fragments_nodes = [
            repre["size"] for repre in representations
        ]
        conditions = torch.tensor([[0] for _ in range(n_samples)], device=device)
        # skipping permutation of indices in reactant state
        xh_fixed = [
            torch.cat(
                [repre[feature_type] for feature_type in FEATURE_MAPPING],
                dim=1,
            )
            for repre in representations
        ]
        print(xh_fixed[2].shape[0])
        print(train_pkl["reactant"]['num_atoms'][train_idx])
        #ground_truth_ts = xh_fixed[1]
        out_samples, out_masks = ddpm_trainer.ddpm.inpaint(
            n_samples=n_samples,
            fragments_nodes=fragments_nodes,
            conditions=conditions,
            return_frames=1,
            resamplings=5,
            jump_length=5,
            timesteps=None,
            xh_fixed=xh_fixed,
            frag_fixed=[0, 2],
        )
        
        # Confirm unchanged reactant state: torch.allclose causing segmentation fault?
        #reactant_diff = torch.allclose(out_samples[0][0] - xh_fixed[0]) #= torch.max(torch.abs(out_samples[0][0] - xh_fixed[0]))
        #print(reactant_diff)
        #assert out_samples[0][0] == xh_fixed[0]
        # Confirm unchanged product state:
        #assert out_samples[0][2] == xh_fixed[2]
        # reactant state (rs_..):
        rs_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=0,
        )
        print(rs_rmsds)
        rs_rmsdsx.append(rs_rmsds[0])

        # transition state (ts_..):
        ts_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=1,
        )
        print(ts_rmsds)
        ts_rmsdsx.append(ts_rmsds[0])
        
        # product state (ps_..):
        ps_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=2,
        )
        print(ps_rmsds)
        ps_rmsdsx.append(ps_rmsds[0])
        #assert len(rmsds) == 1
        print("")

        if output_dir == None:
            continue # skip preparing strings for file output

        # Now wrap up XYZs into an output file with informative comments.
        file_name = f"{rxn_id}.xyz"
        # Reactant state, two versions
        rs_ref_xyz = xyz_block_from_node_features(xh_fixed[0], comment=f"True/calculated reference reactant state.")
        xyz_blocks.append(rs_ref_xyz)
        rs_rec_xyz = xyz_block_from_node_features(out_samples[0][0], comment=f"Reconstructed reactant state. RMSD: {str(round(rs_rmsds[0],6))} Å.")
        xyz_blocks.append(rs_rec_xyz)
        
        # Transition state, two versions
        ts_ref_xyz = xyz_block_from_node_features(xh_fixed[1], comment=f"True/calculated reference transition state.")
        xyz_blocks.append(ts_ref_xyz)
        ts_gen_xyz = xyz_block_from_node_features(out_samples[0][1], comment=f"Generated/inpainted transition state. RMSD: {str(round(ts_rmsds[0],6))} Å.")
        xyz_blocks.append(ts_gen_xyz)

        # Product state, two versions
        ps_ref_xyz = xyz_block_from_node_features(xh_fixed[2], comment=f"True/calculated reference product state.")
        xyz_blocks.append(ps_ref_xyz)
        ps_rec_xyz = xyz_block_from_node_features(out_samples[0][2], comment=f"Reconstructed product state. RMSD: {str(round(ps_rmsds[0],6))} Å.")
        xyz_blocks.append(ps_rec_xyz)

        with open(os.path.join(output_dir, file_name), "w") as f_out:
            f_out.write("\n".join(xyz_blocks))
        
        if i == random_indices[-1]: # no need to keep iterating.
            break

85
92
17
17
[0.0036726165953351302]
[1.0]
[0.004141452406448662]

582
636
17
17
[0.0037444728856587495]
[0.9013558964049089]
[0.003738806495299176]

1030
1131
17
17
[0.003894059423931387]
[0.9174160728493954]
[0.003365289108591928]

14456
16119
17
17
[0.00357805657240741]
[1.0]
[0.0038979561222473235]

18657
20816
17
17
[0.0038386456999949083]
[0.923866825711946]
[0.0034724057271170714]

19738
22024
17
17
[0.004343188175423531]
[1.0]
[0.004071440753370902]

21233
23683
17
17
[0.003649128975228569]
[0.8587585913218475]
[0.0046354548250903795]

27343
30463
17
17
[0.0037816763034445575]
[0.20740817855070914]
[0.004347285023186276]

27677
30832
18
18
[0.0040687243405626025]
[1.0]
[0.0040650315550623374]

32299
35956
18
18
[0.004262057264482316]
[0.46223889155463793]
[0.003767157551638431]

37392
41618
18
18
[0.003812873199168739]
[0.8136124039872683]
[0.005068391659361974]

41209
45847
18
18
[0.0033119217241261907]
[0.19203698038094302]
[0.0038316777724087163]

42208
46957
18
18
[0.0032548

In [19]:
# Report/analysis of RMSDs seen:

rs_mean = round(np.mean(rs_rmsdsx), 6)
rs_std = round(np.std(rs_rmsdsx), 6)
print(f"Reactant state reconstruction RMSD was mean ± std.dev.: \t{rs_mean} \t± {rs_std} Å.")
ts_mean = round(np.mean(ts_rmsdsx), 6)
ts_std = round(np.std(ts_rmsdsx), 6)
print(f"Transition state inpainting RMSD was mean ± std.dev.: \t\t{ts_mean} \t± {ts_std} Å.")
ps_mean = round(np.mean(ps_rmsdsx), 6)
ps_std = round(np.std(ps_rmsdsx), 6)
print(f"Product state reconstruction RMSD was mean ± std.dev.: \t\t{ps_mean} \t± {ps_std} Å.")

Reactant state reconstruction RMSD was mean ± std.dev.: 	0.003754 	± 0.000445 Å.
Transition state inpainting RMSD was mean ± std.dev.: 		0.731946 	± 0.318365 Å.
Product state reconstruction RMSD was mean ± std.dev.: 		0.004029 	± 0.000442 Å.
